In [1]:
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

True

# Models: open and closed source

In [2]:
from langchain_openai import OpenAI

openai_llm = OpenAI(temperature=0.6)

/Users/marcosf/.local/share/virtualenvs/project-R8Px3RHJ/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
text_a = "Q: Qual é a capital do Brasil?\nA:"
text_b = "Q: What is the capital of France?\nA:"

In [4]:
print(text_a, openai_llm.predict(text_a))
print(text_b, openai_llm.predict(text_b))

/Users/marcosf/.local/share/virtualenvs/project-R8Px3RHJ/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:115: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Q: Qual é a capital do Brasil?
A:  A capital do Brasil é Brasília.
Q: What is the capital of France?
A:  The capital of France is Paris.


In [5]:
from langchain_community.llms import HuggingFaceHub

hf_llm = HuggingFaceHub(repo_id="google/flan-t5-base", model_kwargs={"temperature": 0.6})

/Users/marcosf/.local/share/virtualenvs/project-R8Px3RHJ/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/marcosf/.local/share/virtualenvs/project-R8Px3RHJ/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [6]:
print(text_a, hf_llm.predict(text_a))
print(text_b, hf_llm.predict(text_b))

Q: Qual é a capital do Brasil?
A: Brasil
Q: What is the capital of France?
A: Paris


# Prompt Templates and LLMChain

In [7]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
  input_variables=["country"],
  template="Tell me the capital of {country}."
)

prompt_template.format(country="Brazil")

'Tell me the capital of Brazil.'

In [8]:
from langchain.chains import LLMChain

openai_llm_chain = LLMChain(
  llm=openai_llm,
  prompt=prompt_template
)
openai_llm_chain.run(country="Brazil")

/Users/marcosf/.local/share/virtualenvs/project-R8Px3RHJ/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:115: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n\nThe capital of Brazil is Brasília. '

In [9]:
hf_llm_chain = LLMChain(
  llm=hf_llm,
  prompt=prompt_template
)
hf_llm_chain.run(country="Brazil")

'porto alegre'

# Combining multiple chains using Sequential Chain

In [10]:
from langchain.chains import SimpleSequentialChain

In [11]:
base_prompt = PromptTemplate(
  input_variables=["country"],
  template="Tell me the capital of {country}."
)

followup_prompt = PromptTemplate(
  input_variables=["capital"],
  template="List five interesting things to do in {capital}."

)

In [12]:
base_openai_chain = LLMChain(
  llm=openai_llm,
  prompt=base_prompt
)

followup_openai_chain = LLMChain(
  llm=openai_llm,
  prompt=followup_prompt
)

sequential_openai_chain = SimpleSequentialChain(
  chains=[base_openai_chain, followup_openai_chain]
)

sequential_openai_chain.run("Brazil")

"\n\n1. Visit the Three Powers Plaza: This is where the three branches of the Brazilian government are located - the Presidential Palace, the National Congress, and the Supreme Court. It's a great place to learn about Brazil's political system and to see some impressive architecture.\n\n2. Explore the Cathedral of Brasília: Designed by renowned architect Oscar Niemeyer, this unique cathedral is a must-see attraction in Brasília. Its modern and abstract design is a stark contrast to the traditional cathedrals found in other cities.\n\n3. Stroll around the Brasília National Park: This park is a great escape from the hustle and bustle of the city. It's a large green space with walking trails, lakes, and even a small zoo. It's the perfect spot for a picnic or a relaxing afternoon.\n\n4. Visit the National Museum of the Republic: This museum showcases the history and culture of Brazil, with exhibitions on art, archaeology, and anthropology. It's a great place to learn about the country's in

In [13]:
base_hf_chain = LLMChain(
  llm=hf_llm,
  prompt=base_prompt
)

followup_hf_chain = LLMChain(
  llm=hf_llm,
  prompt=followup_prompt
)

sequential_hf_chain = SimpleSequentialChain(
  chains=[base_hf_chain, followup_hf_chain]
)

sequential_hf_chain.run("Brazil")

'Porto alegre is a city in the north of Portugal.'

In [19]:
from langchain.chains import SequentialChain

base_chain_output_key = LLMChain(
  llm=openai_llm,
  prompt=base_prompt,
  output_key="capital"
)

followup_chain_output_key = LLMChain(
  llm=openai_llm,
  prompt=followup_prompt,
  output_key="interesting_things"
)

sequential_chain_output_key = SequentialChain(
  chains=[base_chain_output_key, followup_chain_output_key],
  input_variables=["country"],
  output_variables=["capital", "interesting_things"]
)

sequential_chain_output_key({"country": "Brazil"})

{'country': 'Brazil',
 'capital': '\n\nThe capital of Brazil is Brasília.',
 'interesting_things': '\n\n1. Visit the Presidential Palace: Also known as the Planalto Palace, this impressive building is the official workplace and residence of the President of Brazil. Visitors can take guided tours to learn about the history and architecture of the palace.\n\n2. Explore the National Museum: Located in the heart of Brasília, the National Museum is home to a vast collection of artifacts, artworks, and exhibits that showcase the cultural and natural history of Brazil. It is a must-visit for history buffs and art lovers.\n\n3. Admire the Brasília Cathedral: Designed by renowned architect Oscar Niemeyer, the Brasília Cathedral is an architectural marvel. Its unique circular shape and stunning stained glass windows make it a must-see attraction in the city.\n\n4. Stroll around Lake Paranoá: This artificial lake is a popular spot for outdoor activities and relaxation. Visitors can take a scenic 

# Chatmodels with ChatOpenAI

In [21]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

chat_openai_llm = ChatOpenAI(
  temperature=0.6,
  model="gpt-3.5-turbo"
)

In [23]:
chat_messages = [
  SystemMessage(content="You are a comedian AI assistant. You can tell jokes and make people laugh."),
  HumanMessage(content="What did the tomate tell the lettuce?")
]

chat_openai_llm(chat_messages)

/Users/marcosf/.local/share/virtualenvs/project-R8Px3RHJ/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:115: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


AIMessage(content='Lettuce ketchup!')

# PromptTemplates + LLM + OutputParsers

In [24]:
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser